In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import missingno

import re

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt 

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import Pool, CatBoostClassifier
from sklearn.metrics import r2_score, accuracy_score, precision_score, recall_score, confusion_matrix, precision_recall_curve
from scipy.stats import pearsonr, chi2_contingency
from statsmodels.stats.proportion import proportion_confint

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install catboost
!pip install pandas==0.25.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
data = pd.read_csv('/content/drive/MyDrive/accepted_2007_to_2018q4.csv/accepted_2007_to_2018Q4.csv',
                   parse_dates=['issue_d'], infer_datetime_format=True)

data = data[(data.issue_d >= '2018-01-01 00:00:00') & (data.issue_d < '2019-01-01 00:00:00')]
data = data.reset_index(drop=True)

data.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,130954621,NaN,5000.0,5000.0,5000.0,36 months,20.39,186.82,D,D4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,130964697,NaN,15000.0,15000.0,15000.0,36 months,9.92,483.45,B,B2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,130955326,NaN,11200.0,11200.0,11200.0,60 months,30.79,367.82,G,G1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,130504052,NaN,25000.0,25000.0,25000.0,60 months,21.85,688.35,D,D5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,130956066,NaN,3000.0,3000.0,3000.0,36 months,7.34,93.10,A,A4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
browse_notes = pd.read_excel('/content/drive/MyDrive/LCDataDictionary.xlsx',
                             sheet_name=1)
browse_notes.head()

,BrowseNotesFile,Description
0,acceptD,The date which the borrower accepted the offer
1,accNowDelinq,The number of accounts on which the borrower i...
2,accOpenPast24Mths,Number of trades opened in past 24 months.
3,addrState,The state provided by the borrower in the loan...
4,all_util,Balance to credit limit on all trades


In [9]:
browse_notes['BrowseNotesFile'].dropna().values

array(['acceptD', 'accNowDelinq', 'accOpenPast24Mths', 'addrState',
       'all_util', 'annual_inc_joint', 'annualInc', 'application_type',
       'avg_cur_bal', 'bcOpenToBuy', 'bcUtil', 'chargeoff_within_12_mths',
       'collections_12_mths_ex_med', 'creditPullD', 'delinq2Yrs',
       'delinqAmnt', 'desc', 'dti', 'dti_joint', 'earliestCrLine',
       'effective_int_rate', 'emp_title', 'empLength', 'expD',
       'expDefaultRate', 'ficoRangeHigh', 'ficoRangeLow', 'fundedAmnt',
       'grade', 'homeOwnership', 'id', 'il_util', 'ils_exp_d',
       'initialListStatus', 'inq_fi', 'inq_last_12m', 'inqLast6Mths',
       'installment', 'intRate', 'isIncV', 'listD', 'loanAmnt',
       'max_bal_bc', 'memberId', 'mo_sin_old_rev_tl_op',
       'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mortAcc', 'msa',
       'mths_since_last_major_derog', 'mths_since_oldest_il_open',
       'mths_since_rcnt_il', 'mthsSinceLastDelinq', 'mthsSinceLastRecord',
       'mthsSinceMostRecentInq', 'mthsSinceRecentBc',

In [10]:
browse_feat = browse_notes['BrowseNotesFile'].dropna().values

#### 2) 대출용어 사전과 대출승인 데이터 문자열 규칙 맞추기
  * 대문자, 숫자 앞에 '_' 붙여주고 모두 소문자 변환
    > re.sub('(?<![0-9_])(?=[A-Z0-9])', '_', x).lower()
  * 공백 처리
    > .strip()

In [11]:
browse_feat = [re.sub('(?<![0-9_])(?=[A-Z0-9])', '_', x).lower().strip() for x in browse_feat]

In [12]:
data_feat = data.columns.values

In [13]:
# ar2 에는 없는 ar1의 고유한 값을 반환
np.setdiff1d(data_feat, browse_feat)

array(['collection_recovery_fee', 'debt_settlement_flag',
       'debt_settlement_flag_date', 'deferral_term',
       'disbursement_method', 'funded_amnt_inv', 'hardship_amount',
       'hardship_dpd', 'hardship_end_date', 'hardship_flag',
       'hardship_last_payment_amount', 'hardship_length',
       'hardship_loan_status', 'hardship_payoff_balance_amount',
       'hardship_reason', 'hardship_start_date', 'hardship_status',
       'hardship_type', 'issue_d', 'last_credit_pull_d',
       'last_fico_range_high', 'last_fico_range_low', 'last_pymnt_amnt',
       'last_pymnt_d', 'loan_status', 'mo_sin_old_il_acct',
       'mths_since_recent_bc_dlq', 'mths_since_recent_inq',
       'next_pymnt_d', 'open_act_il',
       'orig_projected_additional_accrued_interest', 'out_prncp',
       'out_prncp_inv', 'payment_plan_start_date', 'policy_code',
       'pymnt_plan', 'recoveries', 'revol_bal_joint',
       'sec_app_chargeoff_within_12_mths',
       'sec_app_collections_12_mths_ex_med', 'sec_ap

In [14]:
np.setdiff1d(browse_feat, data_feat)

array(['accept_d', 'credit_pull_d', 'effective_int_rate', 'exp_d',
       'exp_default_rate', 'ils_exp_d', 'is_inc_v', 'list_d', 'msa',
       'mths_since_most_recent_inq', 'mths_since_oldest_il_open',
       'mths_since_recent_loan_delinq', 'open_il_6m', 'review_status',
       'review_status_d', 'service_fee_rate', 'verified_status_joint'],
      dtype='<U30')

#### 4) 대출 시점(대출용어 사전)에서의 용어 중 대출승인 데이터 컬럼과 같은 의미인 용어를 서로 같게 만든다
  * 대출 시점(대출용어 사전)에서의 용어 중 대출승인 데이터 컬럼과 의미가 같지만 이름이 다른 컬럼들
          ['is_inc_v', 'mths_since_most_recent_inq','mths_since_oldest_il_open','mths_since_recent_loan_delinq', 'verified_status_joint']
  * 대출승인 데이터 컬럼에서의 이름들 (예: verified_status_join → verification_status_joint)
          ['verification_status', 'mths_since_recent_inq', 'mo_sin_old_il_acct','mths_since_recent_bc_dlq', 'verification_status_joint']
  * np.setdiff1d / np.append 사용

In [15]:
wrong = ['is_inc_v', 'mths_since_most_recent_inq', 'mths_since_oldest_il_open',
         'mths_since_recent_loan_delinq', 'verified_status_joint']
correct = ['verification_status', 'mths_since_recent_inq', 'mo_sin_old_il_acct',
           'mths_since_recent_bc_dlq', 'verification_status_joint']

broswse_feat = np.setdiff1d(browse_feat, wrong)
browse_feat = np.append(browse_feat, correct)

#### 5) 대출용어 사전과 대출승인 데이터 컬럼 이름이 같은 것들만 가져온다.
  * np.intersect1d(ar1, ar2) : ar1 과 ar2 의 공통된 항목들만 반환한다

In [17]:

avail_feat = np.intersect1d(browse_feat, data_feat)

X = data[avail_feat].copy()
X.info()

TypeError: ignored